In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [3]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

Handle class imbalance

In [4]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

#### Track Experiments

In [5]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [6]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]

    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)


reports

[{'0': {'precision': 0.9454545454545454,
   'recall': 0.9629629629629629,
   'f1-score': 0.9541284403669725,
   'support': 270.0},
  '1': {'precision': 0.6,
   'recall': 0.5,
   'f1-score': 0.5454545454545454,
   'support': 30.0},
  'accuracy': 0.9166666666666666,
  'macro avg': {'precision': 0.7727272727272727,
   'recall': 0.7314814814814814,
   'f1-score': 0.749791492910759,
   'support': 300.0},
  'weighted avg': {'precision': 0.9109090909090909,
   'recall': 0.9166666666666666,
   'f1-score': 0.91326105087573,
   'support': 300.0}},
 {'0': {'precision': 0.96415770609319,
   'recall': 0.9962962962962963,
   'f1-score': 0.9799635701275046,
   'support': 270.0},
  '1': {'precision': 0.9523809523809523,
   'recall': 0.6666666666666666,
   'f1-score': 0.7843137254901961,
   'support': 30.0},
  'accuracy': 0.9633333333333334,
  'macro avg': {'precision': 0.9582693292370712,
   'recall': 0.8314814814814815,
   'f1-score': 0.8821386478088503,
   'support': 300.0},
  'weighted avg': {'prec

In [ ]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [14]:
import dagshub
dagshub.init(repo_owner='okereolujiechizurum', repo_name='MLFlow', mlflow=True)


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=4ff939ec-7404-444a-a7fc-7d634f9faa0b&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=e6e59b3d2c405fd31550cb175ffe10b4f5f7b2ec6a2c1eaa6435ecde2f44fa60




Accessing as okereolujiechizurum

Initialized MLflow to track repo "okereolujiechizurum/MLFlow"

Repository okereolujiechizurum/MLFlow initialized!

In [17]:
# Initialize MLflow
mlflow.set_experiment("Anomaly Detection 3")
mlflow.set_tracking_uri(uri="https://dagshub.com/okereolujiechizurum/MLFlow.mlflow")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]

    with mlflow.start_run(run_name=model_name):
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy' : report['accuracy'],
            'recall_class_0' : report['0']['recall'],
            'recall_class_1' : report['1']['recall'],
            'f1_score_macro' : report['macro avg']['f1-score'],
        })

        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")


2025/07/05 16:19:55 INFO mlflow.tracking.fluent: Experiment with name 'Anomaly Detection 3' does not exist. Creating a new experiment.
2025/07/05 16:19:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: https://dagshub.com/okereolujiechizurum/MLFlow.mlflow/#/experiments/1/runs/983ef4eae57244869cfd39ef39c24e0d
🧪 View experiment at: https://dagshub.com/okereolujiechizurum/MLFlow.mlflow/#/experiments/1


2025/07/05 16:20:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: https://dagshub.com/okereolujiechizurum/MLFlow.mlflow/#/experiments/1/runs/b6ea99a232964263baba65440e0536d0
🧪 View experiment at: https://dagshub.com/okereolujiechizurum/MLFlow.mlflow/#/experiments/1


2025/07/05 16:20:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: https://dagshub.com/okereolujiechizurum/MLFlow.mlflow/#/experiments/1/runs/aa8dd65a1ca74956a3f73fc077adcc28
🧪 View experiment at: https://dagshub.com/okereolujiechizurum/MLFlow.mlflow/#/experiments/1


2025/07/05 16:20:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: https://dagshub.com/okereolujiechizurum/MLFlow.mlflow/#/experiments/1/runs/cbf4858ffea94c1bbf179c4b99e94b83
🧪 View experiment at: https://dagshub.com/okereolujiechizurum/MLFlow.mlflow/#/experiments/1


### Register the model


In [10]:
model_name = 'XGB-Smote 3'
run_id= input("Please enter the run ID: ")
model_uri = f"runs:/{run_id}/model"

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)
    print(f"Model {model_name} registered successfully.")


Successfully registered model 'XGB-Smote 3'.
2025/07/05 16:05:20 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote 3, version 1


Model XGB-Smote 3 registered successfully.
🏃 View run XGBClassifier With SMOTE at: http://127.0.0.1:5000/#/experiments/218409269353444867/runs/a4089161115d4f7cb2f840c025e049a0
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/218409269353444867


Created version '1' of model 'XGB-Smote 3'.
